## Background
This notebook contains code to test multiple optimization options used by the `optimize` method of the [HistogramRegistration](http://nipy.org/nipy/api/generated/nipy.algorithms.registration.histogram_registration.html) class in `nipy`. The code is tested on resting state fMRI data downloaded from OpenNeuro. The goal is to estimate and apply a standard 6 degree of freedom rigid body transformation to 3D NIfTI data in order to realign an fMRI volume to a template.

## Imports

In [14]:
import nibabel as nib
import time
import numpy as np
from nipy.algorithms.registration import HistogramRegistration, Rigid, resample
from nipy.io.api import save_image

## Load/manipulate data
A single run of a resting state fMRI dataset was downloaded from OpenNeuro. Here, we create separate 3D NIfTI files for the first volume and the 20th volume (arbitrarily selected) of the original timeseries. The first volume is used as the template for realignment. The 20th volume is the one that is to be realigned.

In [7]:
rs_fn = 'ds000256-download/sub-CTS200/func/sub-CTS200_task-restbaseline_bold.nii.gz'
test_template_fn = 'bold_001.nii.gz'
test_imgToRealign_fn = 'bold_020.nii.gz'
rs_img = nib.load(rs_fn)
rs_data = rs_img.get_fdata()

template_nii = nib.nifti1.Nifti1Image(rs_data[:,:,:,0], rs_img.affine, rs_img.header)
imgToRealign_nii = nib.nifti1.Nifti1Image(rs_data[:,:,:,19], rs_img.affine, rs_img.header)

nib.save(template_nii, test_template_fn)
nib.save(imgToRealign_nii, test_imgToRealign_fn)

# Realignment testing script
The `realignmentTest()` function loops through optimization methods `['powell', 'cg', 'bfgs', 'simplex']`, estimates realignment parameters for each, saves the realigned NIfTI file, and logs execution time.

The `estimateMotion()` function does the actual rigid body transform estimation and registration.

Code was adapted from [Jeff MacInnes's pyneal package](https://github.com/jeffmacinnes/pyneal/blob/master/src/pynealPreprocessing.py)

In [27]:
def realignmentTest():
    # opt_methods = ['powell', 'steepest', 'cg', 'bfgs', 'simplex']
    # 'steepest' fails for a reason that I have not investigated yet
    opt_methods = ['powell', 'cg', 'bfgs', 'simplex']
    opt_times = [None] * len(opt_methods)
    mthd_params = np.empty((len(opt_methods), 6))
    mthd_params[:] = None

    # print(opt_times)
    # print(mthd_params)

    # Load images
    niiRefVol = nib.load('bold_001.nii.gz')
    niiVol = nib.load('bold_020.nii.gz')

    for idx, mthd in enumerate(opt_methods):
        print("---------------")
        print("Method {} - {}:".format(idx, mthd))
        start = time.time()
        # Apply realignment
        params, realignedVolume = estimateMotion(niiRefVol, niiVol, mthd)
        end = time.time()
        delta = end - start
        opt_times[idx] = delta
        mthd_params[idx, :] = params
        # print("{} - {} registration time: {}".format(idx, mthd, delta))
        # Save the image as a new nii file (for testing purposes)
        save_image(realignedVolume, 'test_' + mthd + '_bold_020.nii.gz')
        
    print("---------------------")
    print("Registration summary:")
    print("---------------------")
    
    print("\nRegistration time per 'opt' method:\n{}".format(opt_times))
    print("\nRealignment params per 'opt' method:\n{}".format(mthd_params))


def estimateMotion(niiRefVol, niiVol, mthd):
    
    reg = HistogramRegistration(niiVol, niiRefVol, interp='tri')

    # estimate optimal transformation
    T = reg.optimize('rigid', optimizer=mthd)

    # get the realignment parameters:
    rot_x, rot_y, rot_z = np.rad2deg(T.rotation)
    trans_x, trans_y, trans_z = T.translation

    motionParams = np.array([rot_x, rot_y, rot_z, trans_x, trans_y, trans_z])

    realignedVolume = resample(niiVol, T.inv(), niiRefVol)

    return motionParams, realignedVolume

realignmentTest()

---------------
Method 0 - powell:
Initial guess...
translation : [0. 0. 0.]
rotation    : [0. 0. 0.]

Optimizing using fmin_powell
translation : [-0.30057813 -0.20767556  0.381966  ]
rotation    : [0.00381966 0.01       0.01      ]

crl1 = 0.9320968283414985

Optimization terminated successfully.
         Current function value: -0.932097
         Iterations: 1
         Function evaluations: 36
---------------
Method 1 - cg:
Initial guess...
translation : [0. 0. 0.]
rotation    : [0. 0. 0.]

Optimizing using fmin_cg
Optimization terminated successfully.
         Current function value: -0.923707
         Iterations: 0
         Function evaluations: 8
         Gradient evaluations: 1


/Users/jheunis/miniconda3/envs/rtfmri-dev/lib/python3.7/site-packages/nipy/io/files.py:145: FutureWarning: Default `strict` currently False; this will change to True in a future version of nipy
  ni_img = nipy2nifti(img, data_dtype = io_dtype)


---------------
Method 2 - bfgs:
Initial guess...
translation : [0. 0. 0.]
rotation    : [0. 0. 0.]

Optimizing using fmin_bfgs
Optimization terminated successfully.
         Current function value: -0.923707
         Iterations: 0
         Function evaluations: 8
         Gradient evaluations: 1
---------------
Method 3 - simplex:
Initial guess...
translation : [0. 0. 0.]
rotation    : [0. 0. 0.]

Optimizing using fmin
Optimization terminated successfully.
         Current function value: -0.923707
         Iterations: 1
         Function evaluations: 7
---------------------
Registration summary:
---------------------

Registration time per 'opt' method:
[0.4186708927154541, 0.13682198524475098, 0.1139068603515625, 0.11966705322265625]

Realignment params per 'opt' method:
[[ 0.2188504   0.5729578   0.5729578  -0.30057813 -0.20767556  0.381966  ]
 [ 0.          0.          0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.